# TinyMPC Trajectory Tracking Demo

This notebook demonstrates real-time trajectory tracking using the TinyMPC hardware solver with interactive visualization.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import clear_output
import time
import threading
from queue import Queue
import ipywidgets as widgets
from IPython.display import display

# Import TinyMPC components
from simulator import TinyMPCSimulator, ControlMode
from trajectory import TrajectoryFactory, TrajectoryType
from dynamics import LinearizedQuadcopterDynamics, CrazyflieParams, NoiseModel

%matplotlib widget

## Setup Parameters and Initialize System

In [ ]:
# System parameters
control_freq = 50.0  # Hz
simulation_duration = 20.0  # seconds
horizon = 10  # MPC horizon

# Initialize dynamics model
params = CrazyflieParams()
noise_model = NoiseModel()
dynamics = LinearizedQuadcopterDynamics(params, noise_model)

print(f"Dynamics model initialized:")
print(f"  Control frequency: {control_freq} Hz")
print(f"  MPC horizon: {horizon}")
print(f"  Simulation duration: {simulation_duration} s")

## Generate Reference Trajectory

In [ ]:
# Create trajectory selector
trajectory_options = {
    'Circle': TrajectoryType.CIRCLE,
    'Figure-8': TrajectoryType.FIGURE8,
    'Line': TrajectoryType.LINE,
    'Spiral': TrajectoryType.SPIRAL,
    'Waypoints': TrajectoryType.WAYPOINTS,
    'Zigzag': TrajectoryType.ZIGZAG
}

trajectory_selector = widgets.Dropdown(
    options=list(trajectory_options.keys()),
    value='Circle',
    description='Trajectory:'
)

solver_selector = widgets.Dropdown(
    options=['auto', 'hardware', 'software'],
    value='auto',
    description='Solver:'
)

def generate_reference_trajectory(traj_type_str):
    """Generate reference trajectory based on selection"""
    traj_type = trajectory_options[traj_type_str]
    
    # Different parameters for different trajectories
    if traj_type == TrajectoryType.CIRCLE:
        X_ref = TrajectoryFactory.generate_trajectory(
            traj_type, simulation_duration, control_freq,
            radius=1.5, center=[0, 0, 1.2]
        )
    elif traj_type == TrajectoryType.FIGURE8:
        X_ref = TrajectoryFactory.generate_trajectory(
            traj_type, simulation_duration, control_freq,
            scale=1.0, center=[0, 0, 1.2]
        )
    elif traj_type == TrajectoryType.LINE:
        X_ref = TrajectoryFactory.generate_trajectory(
            traj_type, simulation_duration, control_freq,
            start_pos=[0, 0, 1], end_pos=[3, 2, 1.5]
        )
    elif traj_type == TrajectoryType.SPIRAL:
        X_ref = TrajectoryFactory.generate_trajectory(
            traj_type, simulation_duration, control_freq,
            radius_max=1.2, center=[0, 0, 1.0], height_gain=0.8
        )
    elif traj_type == TrajectoryType.WAYPOINTS:
        X_ref = TrajectoryFactory.generate_trajectory(
            traj_type, simulation_duration, control_freq,
            waypoints=[[0, 0, 1], [1.5, 0, 1.2], [1.5, 1.5, 1.5], [0, 1.5, 1.2], [0, 0, 1]]
        )
    elif traj_type == TrajectoryType.ZIGZAG:
        X_ref = TrajectoryFactory.generate_trajectory(
            traj_type, simulation_duration, control_freq,
            amplitude=1.0, frequency=0.3
        )
    else:
        # Default to circle
        X_ref = TrajectoryFactory.generate_trajectory(
            TrajectoryType.CIRCLE, simulation_duration, control_freq,
            radius=1.0, center=[0, 0, 1.2]
        )
    
    return X_ref

# Generate initial trajectory
X_ref = generate_reference_trajectory('Circle')

print(f"Reference trajectory generated:")
print(f"  Shape: {X_ref.shape}")
print(f"  Duration: {X_ref.shape[1] / control_freq:.1f} s")

display(trajectory_selector, solver_selector)

## Initialize MPC Simulator

In [ ]:
def create_simulator(solver_type='auto'):
    """Create MPC simulator with specified solver type"""
    try:
        simulator = TinyMPCSimulator(
            dynamics_model=dynamics,
            X_ref=X_ref,
            horizon=horizon,
            control_mode=ControlMode.TRACKING,
            solver_type=solver_type
        )
        simulator.set_control_frequency(control_freq)
        print(f"Simulator created successfully with {simulator.solver_type} solver")
        return simulator
    except Exception as e:
        print(f"Error creating simulator: {e}")
        return None

simulator = create_simulator('auto')

## Real-time Visualization Setup

In [ ]:
class TrajectoryTracker:
    def __init__(self, simulator, X_ref, control_freq, update_rate=10):
        self.simulator = simulator
        self.X_ref = X_ref
        self.control_freq = control_freq
        self.dt = 1.0 / control_freq
        self.update_rate = update_rate  # Hz for visualization updates
        
        # Simulation state
        self.current_step = 0
        self.max_steps = X_ref.shape[1]
        self.running = False
        
        # Initialize state with some noise
        initial_noise_std = simulator.dynamics_model.noise_model.get_initial_state_noise_std()
        self.current_state = X_ref[:, 0] + np.random.normal(0, initial_noise_std * 0.5, 12)
        
        # History storage
        self.state_history = [self.current_state.copy()]
        self.control_history = []
        self.time_history = [0.0]
        
        # Data queue for thread communication
        self.data_queue = Queue()
        
    def step_simulation(self):
        """Execute one simulation step"""
        if self.current_step >= self.max_steps - 1:
            return False
        
        try:
            # Get reference trajectory for the horizon
            ref_start_idx = min(self.current_step, self.X_ref.shape[1] - 1)
            
            X_ref_horizon = np.zeros((12, horizon))
            U_ref_horizon = np.zeros((4, horizon - 1))
            
            for i in range(horizon):
                ref_idx = min(ref_start_idx + i, self.X_ref.shape[1] - 1)
                X_ref_horizon[:, i] = self.X_ref[:, ref_idx]
            
            # Solve MPC problem
            if self.simulator.solver_type == "hardware":
                solution = self.simulator.mpc.solve(self.current_state, X_ref_horizon.T, U_ref_horizon.T)
                if solution is not None and 'controls' in solution:
                    u_control = solution['controls'][0] if len(solution['controls']) > 0 else np.zeros(4)
                else:
                    u_control = np.zeros(4)
            else:
                # Software solver
                self.simulator.mpc.set_x0(self.current_state)
                self.simulator.mpc.set_x_ref(X_ref_horizon)
                self.simulator.mpc.set_u_ref(U_ref_horizon)
                
                solution = self.simulator.mpc.solve()
                if solution is not None and 'controls' in solution:
                    u_control = solution['controls'].flatten()
                else:
                    u_control = np.zeros(4)
            
            # Add noise and simulate forward
            u_noisy = self._add_actuator_noise(u_control)
            self.current_state = self._simulate_forward(self.current_state, u_noisy)
            
            # Apply constraints
            constraints = self.simulator.constraints
            self.current_state = np.clip(self.current_state, constraints['x_min'], constraints['x_max'])
            
            # Store history
            self.state_history.append(self.current_state.copy())
            self.control_history.append(u_control.copy())
            self.time_history.append((self.current_step + 1) * self.dt)
            
            self.current_step += 1
            return True
            
        except Exception as e:
            print(f"Simulation step error: {e}")
            return False
    
    def _add_actuator_noise(self, u_control):
        """Add actuator noise to control inputs"""
        noise = np.random.normal(0, self.simulator.dynamics_model.noise_model.thrust_noise_std, len(u_control))
        return u_control * (1 + noise)
    
    def _simulate_forward(self, x_current, u_control):
        """Simulate system forward one time step"""
        # Get system matrices
        A, B = self.simulator.A, self.simulator.B
        
        # Add process noise
        process_noise_std = self.simulator.dynamics_model.noise_model.get_state_noise_std(self.dt)
        process_noise = np.random.normal(0, process_noise_std, len(x_current))
        
        # Forward simulation
        gravity_disturbance = getattr(self.simulator.dynamics_model, 'gravity_disturbance', np.zeros(12))
        x_next = A @ x_current + B @ u_control + process_noise + gravity_disturbance
        
        return x_next
    
    def start_simulation(self):
        """Start the simulation loop"""
        self.running = True
        self.simulation_thread = threading.Thread(target=self._simulation_loop)
        self.simulation_thread.daemon = True
        self.simulation_thread.start()
    
    def stop_simulation(self):
        """Stop the simulation loop"""
        self.running = False
    
    def _simulation_loop(self):
        """Main simulation loop running in separate thread"""
        update_interval = 1.0 / self.control_freq
        
        while self.running and self.current_step < self.max_steps - 1:
            start_time = time.time()
            
            # Execute simulation step
            success = self.step_simulation()
            if not success:
                break
            
            # Send data to visualization queue
            data = {
                'step': self.current_step,
                'time': self.time_history[-1],
                'state': self.current_state.copy(),
                'reference': self.X_ref[:, min(self.current_step, self.X_ref.shape[1] - 1)],
                'position_error': np.linalg.norm(self.current_state[:3] - self.X_ref[:3, min(self.current_step, self.X_ref.shape[1] - 1)]),
                'running': self.running
            }
            
            self.data_queue.put(data)
            
            # Maintain real-time execution
            elapsed = time.time() - start_time
            sleep_time = max(0, update_interval - elapsed)
            time.sleep(sleep_time)
        
        # Send final stop signal
        self.data_queue.put({'running': False})
    
    def get_current_data(self):
        """Get current simulation data for visualization"""
        try:
            return self.data_queue.get_nowait()
        except:
            return None

print("TrajectoryTracker class defined")

## Interactive Trajectory Tracking with Real-time Visualization

In [ ]:
# Initialize tracker
tracker = None

def start_tracking():
    global tracker, simulator, X_ref
    
    # Update reference trajectory based on selection
    selected_traj = trajectory_selector.value
    selected_solver = solver_selector.value
    
    X_ref = generate_reference_trajectory(selected_traj)
    simulator = create_simulator(selected_solver)
    
    if simulator is None:
        print("Failed to create simulator")
        return
    
    # Update simulator with new reference
    simulator.X_ref = X_ref
    
    tracker = TrajectoryTracker(simulator, X_ref, control_freq)
    
    print(f"Starting {selected_traj} trajectory tracking with {simulator.solver_type} solver...")
    
    # Create real-time visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Initialize plots
    # 3D trajectory plot
    ax1.set_title('3D Trajectory (XY View)')
    ax1.set_xlabel('X Position (m)')
    ax1.set_ylabel('Y Position (m)')
    ax1.grid(True)
    ax1.axis('equal')
    
    # Reference trajectory (full)
    ax1.plot(X_ref[0, :], X_ref[1, :], 'r--', alpha=0.6, linewidth=2, label='Reference')
    
    # Actual trajectory (will be updated)
    actual_line, = ax1.plot([], [], 'b-', linewidth=2, label='Actual')
    current_pos, = ax1.plot([], [], 'bo', markersize=8, label='Current Position')
    
    ax1.legend()
    
    # Position error over time
    ax2.set_title('Position Error Over Time')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Position Error (m)')
    ax2.grid(True)
    error_line, = ax2.plot([], [], 'g-', linewidth=2)
    
    # Z trajectory
    ax3.set_title('Altitude Tracking')
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Z Position (m)')
    ax3.grid(True)
    
    # Control inputs
    ax4.set_title('Control Inputs')
    ax4.set_xlabel('Time (s)')
    ax4.set_ylabel('Control Signal')
    ax4.grid(True)
    
    plt.tight_layout()
    
    # Storage for plot data
    actual_x, actual_y, actual_z = [], [], []
    error_data = []
    time_data = []
    
    # Status display
    status_text = ax1.text(0.02, 0.98, '', transform=ax1.transAxes, 
                          verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    def update_plots():
        """Update all plots with current data"""
        nonlocal actual_x, actual_y, actual_z, error_data, time_data
        
        data = tracker.get_current_data()
        if data is None:
            return
        
        if not data.get('running', True):
            status_text.set_text('Simulation Complete!')
            return
        
        # Update data
        state = data['state']
        actual_x.append(state[0])
        actual_y.append(state[1])
        actual_z.append(state[2])
        error_data.append(data['position_error'])
        time_data.append(data['time'])
        
        # Update XY trajectory
        actual_line.set_data(actual_x, actual_y)
        current_pos.set_data([state[0]], [state[1]])
        
        # Update bounds
        if len(actual_x) > 1:
            all_x = list(actual_x) + list(X_ref[0, :])
            all_y = list(actual_y) + list(X_ref[1, :])
            margin = 0.2
            ax1.set_xlim(min(all_x) - margin, max(all_x) + margin)
            ax1.set_ylim(min(all_y) - margin, max(all_y) + margin)
        
        # Update error plot
        error_line.set_data(time_data, error_data)
        if len(error_data) > 1:
            ax2.set_xlim(0, max(time_data))
            ax2.set_ylim(0, max(max(error_data) * 1.1, 0.1))
        
        # Update Z plot
        ax3.clear()
        ax3.set_title('Altitude Tracking')
        ax3.set_xlabel('Time (s)')
        ax3.set_ylabel('Z Position (m)')
        ax3.grid(True)
        
        if len(time_data) > 1:
            t_ref = np.linspace(0, max(time_data), len(X_ref[2, :]))
            ax3.plot(t_ref, X_ref[2, :], 'r--', alpha=0.6, label='Reference Z')
            ax3.plot(time_data, actual_z, 'b-', label='Actual Z')
            ax3.legend()
            ax3.set_xlim(0, max(time_data))
        
        # Update control plot
        ax4.clear()
        ax4.set_title('Control Inputs')
        ax4.set_xlabel('Time (s)')
        ax4.set_ylabel('Control Signal')
        ax4.grid(True)
        
        if len(tracker.control_history) > 1:
            controls = np.array(tracker.control_history)
            control_time = time_data[1:]  # Controls are one step behind
            if len(control_time) > 0:
                ax4.plot(control_time, controls[:len(control_time), 0], label='Thrust')
                ax4.plot(control_time, controls[:len(control_time), 1], label='Roll Torque')
                ax4.plot(control_time, controls[:len(control_time), 2], label='Pitch Torque')
                ax4.plot(control_time, controls[:len(control_time), 3], label='Yaw Torque')
                ax4.legend()
                ax4.set_xlim(0, max(control_time))
        
        # Update status
        status_text.set_text(
            f"Step: {data['step']}/{tracker.max_steps}\n"
            f"Time: {data['time']:.2f}s\n"
            f"Error: {data['position_error']:.4f}m\n"
            f"Solver: {simulator.solver_type}"
        )
        
        plt.draw()
    
    # Start simulation
    tracker.start_simulation()
    
    # Real-time update loop
    try:
        while tracker.running and tracker.current_step < tracker.max_steps - 1:
            update_plots()
            plt.pause(0.1)  # Update visualization at 10 Hz
        
        # Final update
        update_plots()
        
        # Calculate final metrics
        if len(error_data) > 0:
            final_error = error_data[-1]
            mean_error = np.mean(error_data)
            max_error = np.max(error_data)
            
            print(f"\nTracking Performance:")
            print(f"  Final error: {final_error:.4f} m")
            print(f"  Mean error: {mean_error:.4f} m")
            print(f"  Max error: {max_error:.4f} m")
            print(f"  Solver: {simulator.solver_type}")
            
    except KeyboardInterrupt:
        print("\nSimulation stopped by user")
        tracker.stop_simulation()
    
    plt.show()

# Control buttons
start_button = widgets.Button(description='Start Tracking', button_style='success')
start_button.on_click(lambda b: start_tracking())

display(start_button)

## Advanced Visualization Options

In [ ]:
def plot_detailed_results():
    """Plot detailed analysis of the tracking results"""
    if tracker is None or len(tracker.state_history) < 2:
        print("No simulation data available. Run tracking first.")
        return
    
    states = np.array(tracker.state_history)
    controls = np.array(tracker.control_history)
    times = np.array(tracker.time_history)
    
    fig, axes = plt.subplots(3, 2, figsize=(15, 18))
    
    # Position tracking in each axis
    for i, (axis_name, ax) in enumerate(zip(['X', 'Y', 'Z'], axes[0])):
        t_ref = np.linspace(0, times[-1], X_ref.shape[1])
        ax.plot(t_ref, X_ref[i, :], 'r--', label=f'Reference {axis_name}', alpha=0.7)
        ax.plot(times, states[:, i], 'b-', label=f'Actual {axis_name}')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel(f'{axis_name} Position (m)')
        ax.set_title(f'{axis_name}-axis Tracking')
        ax.grid(True)
        ax.legend()
    
    # Velocity tracking
    for i, (axis_name, ax) in enumerate(zip(['X', 'Y', 'Z'], axes[1])):
        t_ref = np.linspace(0, times[-1], X_ref.shape[1])
        ax.plot(t_ref, X_ref[6+i, :], 'r--', label=f'Reference V{axis_name.lower()}', alpha=0.7)
        ax.plot(times, states[:, 6+i], 'b-', label=f'Actual V{axis_name.lower()}')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel(f'V{axis_name.lower()} (m/s)')
        ax.set_title(f'{axis_name}-axis Velocity')
        ax.grid(True)
        ax.legend()
    
    # Control inputs
    control_names = ['Thrust', 'Roll Torque', 'Pitch Torque', 'Yaw Torque']
    control_times = times[1:]  # Controls are one step behind
    
    for i in range(min(4, controls.shape[1])):
        ax = axes[2][i//2] if i < 2 else axes[2][1]
        
        if i == 0 or i == 2:  # New subplot
            ax.clear()
        
        ax.plot(control_times[:len(controls)], controls[:, i], 
               label=control_names[i], linewidth=2)
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Control Signal')
        ax.set_title(f'Control Inputs ({"Thrust" if i < 2 else "Torques"})')
        ax.grid(True)
        ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    position_errors = []
    for i in range(len(states)):
        ref_idx = min(i, X_ref.shape[1] - 1)
        error = np.linalg.norm(states[i, :3] - X_ref[:3, ref_idx])
        position_errors.append(error)
    
    print(f"\nDetailed Performance Analysis:")
    print(f"  Simulation duration: {times[-1]:.2f} s")
    print(f"  Total steps: {len(states)}")
    print(f"  Final position error: {position_errors[-1]:.4f} m")
    print(f"  Mean position error: {np.mean(position_errors):.4f} m")
    print(f"  Std position error: {np.std(position_errors):.4f} m")
    print(f"  Max position error: {np.max(position_errors):.4f} m")
    print(f"  RMSE position error: {np.sqrt(np.mean(np.array(position_errors)**2)):.4f} m")

# Analysis button
analysis_button = widgets.Button(description='Show Detailed Analysis', button_style='info')
analysis_button.on_click(lambda b: plot_detailed_results())

display(analysis_button)

## Hardware Performance Comparison

In [ ]:
def compare_solvers():
    """Compare performance between hardware and software solvers"""
    print("Comparing hardware vs software solver performance...")
    
    # Test parameters
    test_duration = 10.0  # Shorter duration for comparison
    test_X_ref = generate_reference_trajectory('Circle')
    
    results = {}
    
    for solver_type in ['software', 'hardware']:
        try:
            print(f"\nTesting {solver_type} solver...")
            
            # Create simulator
            test_simulator = TinyMPCSimulator(
                dynamics_model=dynamics,
                X_ref=test_X_ref,
                horizon=horizon,
                control_mode=ControlMode.TRACKING,
                solver_type=solver_type
            )
            test_simulator.set_control_frequency(control_freq)
            
            # Run simulation
            start_time = time.time()
            test_simulator.simulate(steps=int(test_duration * control_freq), verbose=False)
            total_time = time.time() - start_time
            
            # Get results
            sim_results = test_simulator.get_results()
            
            results[solver_type] = {
                'total_time': total_time,
                'final_error': sim_results['final_position_error'],
                'mean_error': sim_results['mean_position_error'],
                'max_error': sim_results['max_position_error'],
                'rmse_error': sim_results['rmse_position_error']
            }
            
            print(f"  Completed in {total_time:.2f} seconds")
            print(f"  Final error: {sim_results['final_position_error']:.4f} m")
            print(f"  Mean error: {sim_results['mean_error']:.4f} m")
            
        except Exception as e:
            print(f"  {solver_type} solver test failed: {e}")
            results[solver_type] = None
    
    # Display comparison
    if len(results) > 1:
        print(f"\n{'='*50}")
        print("SOLVER COMPARISON RESULTS")
        print(f"{'='*50}")
        
        for solver_type, result in results.items():
            if result is not None:
                print(f"\n{solver_type.upper()} Solver:")
                print(f"  Total simulation time: {result['total_time']:.2f} s")
                print(f"  Final position error: {result['final_error']:.4f} m")
                print(f"  Mean position error: {result['mean_error']:.4f} m")
                print(f"  Max position error: {result['max_error']:.4f} m")
                print(f"  RMSE position error: {result['rmse_error']:.4f} m")
                
                # Calculate real-time factor
                rt_factor = test_duration / result['total_time']
                print(f"  Real-time factor: {rt_factor:.2f}x")
        
        # Speed comparison
        if 'hardware' in results and 'software' in results:
            if results['hardware'] is not None and results['software'] is not None:
                speedup = results['software']['total_time'] / results['hardware']['total_time']
                print(f"\nHardware speedup: {speedup:.2f}x faster than software")

# Comparison button
compare_button = widgets.Button(description='Compare Solvers', button_style='warning')
compare_button.on_click(lambda b: compare_solvers())

display(compare_button)

## Trajectory Gallery

Preview different trajectory types available in the system:

In [ ]:
def show_trajectory_gallery():
    """Display all available trajectory types"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    gallery_duration = 15.0
    
    trajectory_configs = {
        'Circle': (TrajectoryType.CIRCLE, {'radius': 1.2, 'center': [0, 0, 1.2]}),
        'Figure-8': (TrajectoryType.FIGURE8, {'scale': 1.0, 'center': [0, 0, 1.2]}),
        'Line': (TrajectoryType.LINE, {'start_pos': [0, 0, 1], 'end_pos': [2.5, 2, 1.5]}),
        'Spiral': (TrajectoryType.SPIRAL, {'radius_max': 1.0, 'center': [0, 0, 1.0], 'height_gain': 0.8}),
        'Waypoints': (TrajectoryType.WAYPOINTS, {'waypoints': [[0, 0, 1], [1, 0, 1.2], [1, 1, 1.5], [0, 1, 1.2], [0, 0, 1]]}),
        'Zigzag': (TrajectoryType.ZIGZAG, {'amplitude': 0.8, 'frequency': 0.4})
    }
    
    for i, (name, (traj_type, params)) in enumerate(trajectory_configs.items()):
        ax = axes[i]
        
        # Generate trajectory
        X_traj = TrajectoryFactory.generate_trajectory(
            traj_type, gallery_duration, control_freq, **params
        )
        
        # Plot XY view
        ax.plot(X_traj[0, :], X_traj[1, :], 'b-', linewidth=2)
        ax.plot(X_traj[0, 0], X_traj[1, 0], 'go', markersize=8, label='Start')
        ax.plot(X_traj[0, -1], X_traj[1, -1], 'ro', markersize=8, label='End')
        
        ax.set_title(f'{name} Trajectory', fontsize=12, fontweight='bold')
        ax.set_xlabel('X Position (m)')
        ax.set_ylabel('Y Position (m)')
        ax.grid(True, alpha=0.3)
        ax.axis('equal')
        ax.legend()
        
        # Add trajectory info
        path_length = np.sum(np.sqrt(np.diff(X_traj[0, :])**2 + np.diff(X_traj[1, :])**2 + np.diff(X_traj[2, :])**2))
        z_range = (np.min(X_traj[2, :]), np.max(X_traj[2, :]))
        
        info_text = f"Length: {path_length:.1f}m\nZ: {z_range[0]:.1f}-{z_range[1]:.1f}m"
        ax.text(0.02, 0.98, info_text, transform=ax.transAxes, 
               verticalalignment='top', fontsize=9,
               bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue', alpha=0.7))
    
    plt.tight_layout()
    plt.suptitle('Trajectory Gallery - Available Patterns', fontsize=16, y=0.98)
    plt.show()
    
    print("Trajectory Gallery:")
    for name, (traj_type, params) in trajectory_configs.items():
        print(f"  {name}: {params}")

# Gallery button
gallery_button = widgets.Button(description='Show Trajectory Gallery', button_style='primary')
gallery_button.on_click(lambda b: show_trajectory_gallery())

display(gallery_button)

## Summary

This notebook provides:

1. **Real-time trajectory tracking** with hardware/software MPC solver
2. **Interactive visualization** showing reference vs actual trajectories
3. **Multiple trajectory types** (circle, figure-8, line, spiral, waypoints, zigzag)
4. **Performance analysis** and solver comparison
5. **Live monitoring** of position error and control inputs

Use the controls above to:
- Select different trajectory types and solver configurations
- Start real-time tracking simulation
- Analyze detailed performance results
- Compare hardware vs software solver performance
- Browse available trajectory patterns

The demo showcases the TinyMPC hardware acceleration capabilities for real-time quadcopter control.